In [1]:
import sys, csv
from modules.wordTokenizer import WordTokenizer
from modules.charTokenizer import CharTokenizer
from modules.LSTMCRFNetwork import LSTMCRFNetwork
import numpy as np

Using TensorFlow backend.
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type

In [2]:
MAX_TEXTS = 1000000
#Read product file
texts, tags, brands = [], [], []
with open("data/train_products.csv", 'r') as f:
    reader = csv.DictReader(f, fieldnames=["title","brand","tags"])
    count = 0
    for row in reader:
        #print(row)
        count += 1
        text, tag_set = row['title'], row['tags'].split(' ')[:-1]
        texts.append(text)
        tags.append(tag_set)
        brands.append(row['brand'])
        if count >= MAX_TEXTS:
            break

In [3]:
#Create training examples
numSentences = len(texts)
np.random.seed(0)
training_examples = np.random.binomial(1, 0.7, numSentences)

In [4]:
#Separate data in training and test
trainData = []
testData  = []

nerLabels_train =[]
nerLabels_test = []


for example in range(numSentences):
    if training_examples[example] == 1:
        trainData.append(texts[example])
        nerLabels_train.append(tags[example])
    else:
        testData.append(texts[example])
        nerLabels_test.append(tags[example])

In [5]:
#create subset of train data
numTrainSentences = 1000

trainData_tiny       = trainData[0:numTrainSentences]
nerLabels_train_tiny = nerLabels_train[0:numTrainSentences]

In [6]:
#Initiate network
network = LSTMCRFNetwork()

In [7]:
#Create tokenized sentences
wordTokenizer = WordTokenizer(30,'models/word_tokenizer')
wordTokenizer.load()
data = wordTokenizer.tokenize(trainData_tiny)
testData = wordTokenizer.tokenize(testData)

In [8]:
#Generate pad labels for network
labels = network.get_labels(nerLabels_train_tiny,wordTokenizer)
testLabels = network.get_labels(nerLabels_test,wordTokenizer)

Getting labels...
Getting labels...


In [9]:
network.compile(wordTokenizer)
network.train(data, labels, epochs=8, validation_split=0.5, batch_size=32)

Loading spanish embedding...


W0802 04:51:12.679719 140025069836096 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 04:51:12.702478 140025069836096 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 04:51:12.705120 140025069836096 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 04:51:12.715055 140025069836096 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final

Found 855381 word vectors.
Creating embedding layer...
Creating network...


W0802 04:51:16.408431 140025069836096 deprecation.py:323] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprec

Compiling network...
Training...
Train on 500 samples, validate on 500 samples
Epoch 1/8
500/500 [==============================] - 7s 14ms/step - loss: 5.6283 - crf_viterbi_accuracy: 0.7057 - val_loss: 5.4915 - val_crf_viterbi_accuracy: 0.7849

Epoch 00001: saving model to models/lstmCRF.h5
Epoch 2/8
500/500 [==============================] - 4s 8ms/step - loss: 5.4231 - crf_viterbi_accuracy: 0.7979 - val_loss: 5.3973 - val_crf_viterbi_accuracy: 0.7898

Epoch 00002: saving model to models/lstmCRF.h5
Epoch 3/8
500/500 [==============================] - 4s 8ms/step - loss: 5.3397 - crf_viterbi_accuracy: 0.8185 - val_loss: 5.3412 - val_crf_viterbi_accuracy: 0.8207

Epoch 00003: saving model to models/lstmCRF.h5
Epoch 4/8
500/500 [==============================] - 4s 8ms/step - loss: 5.2757 - crf_viterbi_accuracy: 0.8410 - val_loss: 5.2952 - val_crf_viterbi_accuracy: 0.8388

Epoch 00004: saving model to models/lstmCRF.h5
Epoch 5/8
500/500 [==============================] - 4s 9ms/step - l

In [10]:
network.evaluate(testData,testLabels)

Evaluating...
18558/18558 [==============================] - 6s 318us/step
              precision    recall  f1-score   support

           O   0.981298  0.992244  0.986741    532625
         B-B   0.700469  0.587279  0.638899     18584
         I-B   0.839568  0.393600  0.535943      5531

    accuracy                       0.972779    556740
   macro avg   0.840445  0.657708  0.720528    556740
weighted avg   0.970516  0.972779  0.970651    556740

